# 2020语言与智能技术竞赛：机器阅读理解任务
https://aistudio.baidu.com/aistudio/competition/detail/28

机器阅读理解 (MRC, Machine Reading Comprehension) 是指让机器阅读文本，然后回答和阅读内容相关的问题。阅读理解是自然语言处理和人工智能领域的重要前沿课题，对于提升机器的智能水平、使机器具有持续知识获取的能力等具有重要价值，近年来受到学术界和工业界的广泛关注。

中国中文信息学会(CCF, the China Computer Federation)、中国计算机学会(CIPS, Chinese Information Processing Society of China)和百度公司已经于2018和2019年连续联合举办了机器阅读理解评测，极大地推动了中文机器阅读理解技术的发展。随着技术的进步，当前的一些模型已经能够在一些阅读理解测试集上取得较好的性能。但在实际应用中，这些模型所表现出的鲁棒性仍然较差。因此，“2020 语言与智能技术竞赛”将继续举办机器阅读理解任务的评测，重点关注阅读理解模型在真实应用场景中的鲁棒性，挑战模型的过敏感性、过稳定性以及泛化能力等。

本次评测将提供面向真实应用场景的高质量中文阅读理解数据集DuReader Robust，旨在为研究者和开发者提供学术和技术交流的平台， 进一步提升机器阅读理解的研究水平，推动语言理解和人工智能领域技术和应用的发展。本次竞赛将在第五届“语言与智能高峰论坛”举办技术交流论坛和颁奖仪式。 诚邀学术界和工业界的研究者和开发者参加本次竞赛！

## 赛程安排
    2020/3/10 	启动竞赛报名，发放样例数据
    2020/3/31 	开放评测入口和排行榜，对报名者发放全部训练数据和第一批测试数据
    2020/5/12 	报名截止
    2020/5/13 	发放最终测试数据
    2020/5/20 	系统结果提交截止
    2020/5/30 	公布竞赛结果，接收系统报告和论文
    2020/6/30 	论文提交截止日期
    2020/7 	在“语言与智能高峰论坛”上交流和颁奖

## 数据介绍 Data

本次竞赛数据集共包含约21K问题，其中包括15K训练集，约1.4K领域内开发集和5K测试集。测试集包含了领域内测试集和鲁棒性测试集，其中鲁棒性测试集包括了过敏感测试集、过稳定测试集以及泛化能力测试集。全部数据集将分为4个部分供参赛用户下载：

1.训练集：共15K样本，用于竞赛模型训练。
2.开发集：共1.4K样本，包含答案，用于竞赛模型训练和参数调试。
3.测试集1：共2K个样本，主要包含了大部分领域内测试集和少部分鲁棒性测试集，不提供参考答案，用于参赛者在比赛平台上自助验证模型效果。为了防止针对测试集的调试，数据中将会额外加入混淆数据。
4.测试集2：是本次竞赛最终测试数据（含测试集1），共5K问题，包含全部领域内测试集和鲁棒性测试集，不提供参考答案。为了防止针对测试集的调试，数据中将会额外加入混淆数据。该部分数据结果不能在比赛平台上自助验证。
### 数据样本 Data Sample

平台提供的数据为JSON文件格式，样例如下:

    {
        "data": [
            {
                "paragraphs": [
                    {
                        "qas": [
                            {
                                "question": "非洲气候带", 
                                "id": "bd664cb57a602ae784ae24364a602674", 
                                "answers": [
                                    {
                                        "text": "热带气候", 
                                        "answer_start": 45
                                    }
                                ]
                            }
                        ], 
                        "context": "1、全年气温高，有热带大陆之称。主要原因在与赤道穿过大陆中部，位于南北纬30度之间，主要是热带气候，没有温带和寒带… 
                    }, 
                    {
                        "qas": [
                            {
                                "question": "韩国全称", 
                                "id": "a7eec8cf0c55077e667e0d85b45a6b34", 
                                "answers": [
                                    {
                                        "text": "大韩民国", 
                                        "answer_start": 5
                                    }
                                ]
                            }
                        ], 
                        "context": "韩国全称“大韩民国”，位于朝鲜半岛南部，隔“三八线”与朝鲜民主主义人民共和国相邻，面积9.93万平方公理… "
                    }
                ], 
                "title": ""
            }
        ]
    }





* 百度LIC2020的机器阅读理解赛道，非官方baseline
* 直接用RoBERTa+Softmax预测首尾
* BASE模型在第一期测试集上能达到0.69的F1，优于官方baseline
* 如果你显存足够，可以换用RoBERTa Large模型，F1可以到0.71

In [1]:
# !pip install bert4keras

In [2]:
!nvidia-smi

/bin/sh: 1: nvidia-smi: not found


In [13]:
import json, os
import numpy as np
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Layer, Dense, Permute, Bidirectional, GRU
from keras.models import Model
from tqdm import tqdm

# 基本信息
maxlen = 128
epochs = 20
batch_size = 4
learing_rate = 2e-5

# 下载模型

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

## 解压

In [5]:
# !ls /content/drive/"My Drive"/kaikeba/project03/roberta/data
!ls ./dureader_robust-data

License.docx  __pycache__  dev.json	       evaluate.py  output
README.md     demo	   dev.json.pred.json  model	    train.json


In [6]:
# ! unzip -o /content/drive/"My Drive"/kaikeba/project03/roberta/data/chinese_roberta_wwm_large_ext_L-24_H-1024_A-16.zip -d /content/drive/"My Drive"/kaikeba/project03/roberta/data
import os
import zipfile
already_ziped=True
if not already_ziped:
    zip_file='./dureader_robust-data/model/chinese_roberta_wwm_base_ext.zip'
    zf=zipfile.ZipFile(zip_file)
    zf.extractall('./dureader_robust-data/model/roberta_wwm_base')
    zf.close()

## 设置数据路径

In [14]:
# data_dir='/content/drive/My Drive/kaikeba/project03/roberta/data'
# output_dir='/content/drive/My Drive/kaikeba/project03/roberta/output'
data_dir='./dureader_robust-data'
output_dir='./dureader_robust-data/output'

# 模型路径

## bert

In [15]:
bert_dir = './dureader_robust-data/model/roberta_wwm_base'
config_path = f'{bert_dir}/bert_config.json'
checkpoint_path = f'{bert_dir}/bert_model.ckpt'
dict_path = f'{bert_dir}/vocab.txt'

# 加载数据

In [9]:
def load_data(filename):
    D = []
    for d in json.load(open(filename))['data'][0]['paragraphs']:
        for qa in d['qas']:
            D.append([
                qa['id'], d['context'], qa['question'],
                [a['text'] for a in qa.get('answers', [])]
            ])
    return D

# 读取数据

In [10]:
train_data = load_data(
    # os.path.join(data_dir,'train.json')
    os.path.join(data_dir,'demo/demo_train.json')
)

In [11]:
train_data[0]

['bd664cb57a602ae784ae24364a602674',
 '1、全年气温高，有热带大陆之称。主要原因在与赤道穿过大陆中部，位于南北纬30度之间，主要是热带气候，没有温带和寒带。2、气候带呈明显带状分布，且南北对称。原因在于赤道穿过大陆中部，整个大陆基本被赤道均分为两部分。因此，纬度地带性明显。气候带以热带雨林为中心，向南北依次分布着热带草原、热带沙漠和地中海式气候。3、气候炎热干燥。第一：热带雨林气候面积较小，主要位于刚果河流域，面积较小。第二，地中海式气候，位于大陆的南北边缘，面积较小。夏季炎热而干旱，冬季温暖而湿润。第三，面积较大热带草原气候，有明显的干湿季。第四，热带沙漠气候主要位于撒哈拉大沙漠和西南角狭长地带。而撒哈拉沙漠占非洲总面积的1/4，全年炎热干燥，日照时间长，昼夜温差大。总之，全非洲纬度低，气温高；干燥地区广，常年湿润地区面积小。',
 '非洲气候带',
 ['热带气候']]

# 建立分词器

In [12]:
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 子串搜索

In [13]:
def search(pattern, sequence):
    """从sequence中寻找子串pattern
    如果找到，返回第一个下标；否则返回-1。
    """
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1

# 数据生成器

In [14]:
class data_generator(DataGenerator):
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, item in self.sample(random):
            context, question, answers = item[1:]
            token_ids, segment_ids = tokenizer.encode(
                question, context, maxlen=maxlen
            )
            a = np.random.choice(answers)
            a_token_ids = tokenizer.encode(a)[0][1:-1]
            start_index = search(a_token_ids, token_ids)
            if start_index != -1:
                labels = [[start_index], [start_index + len(a_token_ids) - 1]]
                batch_token_ids.append(token_ids)
                batch_segment_ids.append(segment_ids)
                batch_labels.append(labels)
                if len(batch_token_ids) == self.batch_size or is_end:
                    batch_token_ids = sequence_padding(batch_token_ids)
                    batch_segment_ids = sequence_padding(batch_segment_ids)
                    batch_labels = sequence_padding(batch_labels)
                    yield [batch_token_ids, batch_segment_ids], batch_labels
                    batch_token_ids, batch_segment_ids, batch_labels = [], [], []

# Mask

In [17]:
class MaskedSoftmax(Layer):
    """
    在序列长度那一维进行softmax，并mask掉padding部分
    """
    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask, 2)
            inputs = inputs - (1.0 - mask) * 1e12
        return K.softmax(inputs, 1)

# 构建模型

In [18]:
model = build_transformer_model(
    config_path,
    checkpoint_path,
)

output = Bidirectional(GRU(384,return_sequences=True))(model.output) 
output = Dense(2)(output)
output = MaskedSoftmax()(output)
output = Permute((2, 1))(output)
# gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(15,return_sequences=True))(inputs)
model = Model(model.input, output)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
____________________________________________________________________________________________

# 评估函数

In [17]:
def sparse_categorical_crossentropy(y_true, y_pred):
    # y_true需要重新明确一下shape和dtype
    y_true = K.reshape(y_true, K.shape(y_pred)[:-1])
    y_true = K.cast(y_true, 'int32')
    y_true = K.one_hot(y_true, K.shape(y_pred)[2])
    # 计算交叉熵
    return K.mean(K.categorical_crossentropy(y_true, y_pred))


def sparse_accuracy(y_true, y_pred):
    # y_true需要重新明确一下shape和dtype
    y_true = K.reshape(y_true, K.shape(y_pred)[:-1])
    y_true = K.cast(y_true, 'int32')
    # 计算准确率
    y_pred = K.cast(K.argmax(y_pred, axis=2), 'int32')
    return K.mean(K.cast(K.equal(y_true, y_pred), K.floatx()))

# 编译模型

In [18]:
model.compile(
    loss=sparse_categorical_crossentropy,
    optimizer=Adam(learing_rate),
    metrics=[sparse_accuracy]
)

# 答案抽取

In [19]:
def extract_answer(question, context, max_a_len=16):
    """
    抽取答案函数
    """
    max_q_len = 64
    q_token_ids = tokenizer.encode(question, maxlen=max_q_len)[0]
    c_token_ids = tokenizer.encode(
        context, maxlen=maxlen - len(q_token_ids) + 1
    )[0]
    token_ids = q_token_ids + c_token_ids[1:]
    segment_ids = [0] * len(q_token_ids) + [1] * (len(c_token_ids) - 1)
    c_tokens = tokenizer.tokenize(context)[1:-1]
    mapping = tokenizer.rematch(context, c_tokens)
    probas = model.predict([[token_ids], [segment_ids]])[0]
    probas = probas[:, len(q_token_ids):-1]
    start_end, score = None, -1
    for start, p_start in enumerate(probas[0]):
        for end, p_end in enumerate(probas[1]):
            if end >= start and end < start + max_a_len:
                if p_start * p_end > score:
                    start_end = (start, end)
                    score = p_start * p_end
    start, end = start_end
    return context[mapping[start][0]:mapping[end][-1] + 1]

# 预测文件生成

In [20]:
def predict_to_file(infile, out_file):
    """预测结果到文件，方便提交
    """
    fw = open(out_file, 'w', encoding='utf-8')
    R = {}
    for d in tqdm(load_data(infile)):
        a = extract_answer(d[2], d[1])
        R[d[0]] = a
    R = json.dumps(R, ensure_ascii=False, indent=4)
    fw.write(R)
    fw.close()

# 官方评估函数

In [21]:
import sys
import io
import json
# sys.path.append('/content/drive/My Drive/kaikeba/project03/roberta/data')
sys.path.append('./dureader_robust-data')
from evaluate import evaluate as src_evaluate
from collections import OrderedDict

In [22]:
def evaluate(filename):
    """
    评测函数（官方提供评测脚本evaluate.py）
    """
    predict_to_file(filename, filename + '.pred.json')
    ref_ans = json.load(io.open(filename))
    pred_ans = json.load(io.open(filename + '.pred.json'))
    F1, EM, TOTAL, SKIP = src_evaluate(ref_ans, pred_ans)
    output_result = OrderedDict()
    output_result['F1'] = '%.3f' % F1
    output_result['EM'] = '%.3f' % EM
    output_result['TOTAL'] = TOTAL
    output_result['SKIP'] = SKIP
    return output_result


class Evaluator(keras.callbacks.Callback):
    """
    评估和保存模型
    """
    def __init__(self):
        self.best_val_f1 = 0.

    def on_epoch_end(self, epoch, logs=None):
        metrics = evaluate(
            os.path.join(data_dir,'dev.json')
            # os.path.join(data_dir,'demo_dev.json')
        )
        if float(metrics['F1']) >= self.best_val_f1:
            self.best_val_f1 = float(metrics['F1'])
            model.save_weights(os.path.join(output_dir,'roberta_best_model.weights'))
            model.save(os.path.join(output_dir,'roberta_best_model.h5'))
        metrics['BEST_F1'] = self.best_val_f1
        print(metrics)

# 获取数据

In [23]:
train_generator = data_generator(train_data, batch_size)
evaluator = Evaluator()

# 模型训练

In [24]:
epochs=5
model.fit_generator(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    verbose=1,
    callbacks=[evaluator]
)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
25/25 [==============================] - 124s 5s/step - loss: 3.7451 - sparse_accuracy: 0.1900


100%|██████████| 1417/1417 [13:45<00:00,  1.72it/s]


OrderedDict([('F1', '39.997'), ('EM', '24.559'), ('TOTAL', 1417), ('SKIP', 0), ('BEST_F1', 39.997)])
Epoch 2/5
25/25 [==============================] - 83s 3s/step - loss: 1.2948 - sparse_accuracy: 0.7150


100%|██████████| 1417/1417 [13:53<00:00,  1.70it/s]


...

OrderedDict([('F1', '50.341'), ('EM', '35.498'), ('TOTAL', 1417), ('SKIP', 0), ('BEST_F1', 50.341)])
Epoch 4/5
25/25 [==============================] - 83s 3s/step - loss: 0.1030 - sparse_accuracy: 0.9950


100%|██████████| 1417/1417 [14:58<00:00,  1.58it/s]


OrderedDict([('F1', '51.545'), ('EM', '37.121'), ('TOTAL', 1417), ('SKIP', 0), ('BEST_F1', 51.545)])
Epoch 5/5
25/25 [==============================] - 83s 3s/step - loss: 0.0483 - sparse_accuracy: 1.0000


100%|██████████| 1417/1417 [14:50<00:00,  1.59it/s]


OrderedDict([('F1', '52.402'), ('EM', '37.615'), ('TOTAL', 1417), ('SKIP', 0), ('BEST_F1', 52.402)])


# 加载最优模型

In [25]:
from keras.models import load_model
model=load_model(os.path.join(output_dir,'roberta_best_model.h5'),custom_objects={'MaskedSoftmax':MaskedSoftmax,'sparse_accuracy':sparse_accuracy})
print(evaluate(os.path.join(data_dir,'dev.json')))

100%|██████████| 1417/1417 [18:13<00:00,  1.30it/s]


OrderedDict([('F1', '52.402'), ('EM', '37.615'), ('TOTAL', 1417), ('SKIP', 0)])


In [26]:
# from keras.models import load_model
# model=load_model(os.path.join(output_dir,'roberta_best_model.h5'))

In [27]:
# model.load_weights(os.path.join(data_dir,'best_model.weights'))

# 预测结果

## Test1

In [30]:
predict_to_file(os.path.join(data_dir,'dev.json'), os.path.join(output_dir,'dev.pred1.json'))


 21%|██▏       | 303/1417 [05:48<26:26,  1.42s/it]


 43%|████▎     | 605/1417 [11:20<13:21,  1.01it/s]


 64%|██████▍   | 907/1417 [15:52<06:47,  1.25it/s]


 85%|████████▌ | 1205/1417 [20:19<03:05,  1.14it/s]


100%|██████████| 1417/1417 [23:17<00:00,  1.01it/s]


## Test2

In [ ]:
predict_to_file(os.path.join(data_dir,'test2.json'),  os.path.join(output_dir,'pred2.json'))

# 保存上传结果

# bert4keras
- Our light reimplement of bert for keras
- 更清晰、更轻量级的keras版bert
- 个人博客：https://kexue.fm/
- 在线文档：http://bert4keras.spaces.ac.cn/ （还在构建中）

## 说明
这是笔者重新实现的keras版的bert，致力于用尽可能清爽的代码来实现结合bert和keras。

本项目的初衷是为了修改、定制上的方便，所以可能会频繁更新。

因此欢迎star，但不建议fork，因为你fork下来的版本可能很快就过期了。

## 功能
目前已经实现：
- 加载bert/roberta/albert的预训练权重进行finetune；
- 实现语言模型、seq2seq所需要的attention mask；
- 丰富的<a href="https://github.com/bojone/bert4keras/tree/master/examples">examples</a>；
- 从零预训练代码（支持TPU、多GPU，请看<a href="https://github.com/bojone/bert4keras/tree/master/pretraining">pretraining</a>）；
- 兼容keras、tf.keras

## 使用
安装稳定版：
```shell
pip install bert4keras
```
安装最新版：
```shell
pip install git+https://www.github.com/bojone/bert4keras.git
```

使用例子请参考<a href="https://github.com/bojone/bert4keras/blob/master/examples">examples</a>目录。

之前基于keras-bert给出的<a href="https://github.com/bojone/bert_in_keras">例子</a>，仍适用于本项目，只需要将`bert_model`的加载方式换成本项目的。

理论上兼容Python2和Python3，实验环境是Python 2.7、Tesorflow 1.14+以及Keras 2.3.1（已经在2.2.4、2.3.0、2.3.1、tf.keras下测试通过）。

当然，乐于贡献的朋友如果发现了某些bug的话，也欢迎指出修正甚至Pull Requests～

## 权重

目前支持加载的权重：
- <strong>Google原版bert</strong>: https://github.com/google-research/bert
- <strong>brightmart版roberta</strong>: https://github.com/brightmart/roberta_zh
- <strong>哈工大版roberta</strong>: https://github.com/ymcui/Chinese-BERT-wwm
- <strong>Google原版albert</strong><sup><a href="https://github.com/bojone/bert4keras/issues/29#issuecomment-552188981">[例子]</a></sup>: https://github.com/google-research/ALBERT
- <strong>brightmart版albert</strong>: https://github.com/brightmart/albert_zh
- <strong>转换后的albert</strong>: https://github.com/bojone/albert_zh
- <strong>华为的NEZHA</strong>: https://github.com/huawei-noah/Pretrained-Language-Model/tree/master/NEZHA-TensorFlow
- <strong>自研语言模型</strong>: https://github.com/ZhuiyiTechnology/pretrained-models
- <strong>T5模型</strong>: https://github.com/google-research/text-to-text-transfer-transformer
- <strong>GPT2_ML</strong>: https://github.com/imcaspar/gpt2-ml
- <strong>Google原版ELECTRA</strong>: https://github.com/google-research/electra
- <strong>哈工大版ELECTRA</strong>: https://github.com/ymcui/Chinese-ELECTRA
- <strong>CLUE版ELECTRA</strong>: https://github.com/CLUEbenchmark/ELECTRA

<strong>注意事项</strong>
- 注1：brightmart版albert的开源时间早于Google版albert，这导致早期brightmart版albert的权重与Google版的不完全一致，换言之两者不能直接相互替换。为了减少代码冗余，bert4keras的0.2.4及后续版本均只支持加载<u>Google版</u>以brightmart版中<u>带Google字眼</u>的权重。如果要加载早期版本的权重，请用<a href="https://github.com/bojone/bert4keras/releases/tag/v0.2.3">0.2.3版本</a>，或者考虑作者转换过的<a href="https://github.com/bojone/albert_zh">albert_zh</a>。
- 注2：下载下来的ELECTRA权重，如果没有json配置文件的话，参考<a href="https://github.com/ymcui/Chinese-ELECTRA/issues/3">这里</a>自己改一个。

## 更新
- <strong>2020.04.29</strong>: 增加重计算（参考<a href="https://github.com/bojone/keras_recompute">keras_recompute</a>），可以通过时间换空间，通过设置环境变量`RECOMPUTE=1`启用。
- <strong>2020.04.25</strong>: 优化tf2下的表现。
- <strong>2020.04.16</strong>: 所有example均适配tensorflow 2.0。
- <strong>2020.04.06</strong>: 增加UniLM预训练模式（测试中）。
- <strong>2020.04.06</strong>: 完善`rematch`方法。
- <strong>2020.04.01</strong>: `Tokenizer`增加`rematch`方法，给出分词结果与原序列的映射关系。
- <strong>2020.03.30</strong>: 尽量统一py文件的写法。
- <strong>2020.03.25</strong>: 支持ELECTRA。
- <strong>2020.03.24</strong>: 继续加强`DataGenerator`，允许传入迭代器时进行局部shuffle。
- <strong>2020.03.23</strong>: 增加调整Attention的`key_size`的选项。
- <strong>2020.03.17</strong>: 增强`DataGenerator`；优化模型写法。
- <strong>2020.03.15</strong>: 支持<a href="https://github.com/imcaspar/gpt2-ml">GPT2_ML</a>。
- <strong>2020.03.10</strong>: 支持Google的<a href="https://github.com/google-research/text-to-text-transfer-transformer">T5</a>模型。
- <strong>2020.03.05</strong>: 将`tokenizer.py`更名为`tokenizers.py`。
- <strong>2020.03.05</strong>: `application='seq2seq'`改名为`application='unilm'`。
- <strong>2020.03.05</strong>: `build_bert_model`更名为`build_transformer_model`。
- <strong>2020.03.05</strong>: 重写`models.py`结构。
- <strong>2020.03.04</strong>: 将`bert.py`更名为`models.py`。
- <strong>2020.03.02</strong>: 重构mask机制（用回Keras自带的mask机制），以便更好地编写更复杂的应用。
- <strong>2020.02.22</strong>: 新增`AutoRegressiveDecoder`类，统一处理Seq2Seq的解码问题。
- <strong>2020.02.19</strong>: transformer block的前缀改为Transformer（本来是Encoder），使得其含义局限性更少。
- <strong>2020.02.13</strong>: 优化`load_vocab`函数；将`build_bert_model`中的`keep_words`参数更名为`keep_tokens`，此处改动可能会对部分脚本产生影响。
- <strong>2020.01.18</strong>: 调整文本处理方式，去掉codecs的使用。
- <strong>2020.01.17</strong>: 各api日趋稳定，为了方便大家使用，打包到<a href="https://pypi.org/project/bert4keras/">pypi</a>，首个打包版本号为0.4.6。
- <strong>2020.01.10</strong>: 重写模型mask方案，某种程度上让代码更为简练清晰；后端优化。
- <strong>2019.12.27</strong>: 重构预训练代码，减少冗余；目前支持RoBERTa和GPT两种预训练方式，详见<a href="https://github.com/bojone/bert4keras/tree/master/pretraining/">pretraining</a>。
- <strong>2019.12.17</strong>: 适配华为的<a href="https://github.com/huawei-noah/Pretrained-Language-Model/tree/master/NEZHA">nezha</a>权重，只需要在`build_bert_model`函数里加上`model='nezha'`；此外原来albert的加载方式`albert=True`改为`model='albert'`。
- <strong>2019.12.16</strong>: 通过跟keras 2.3+版本类似的思路给低版本引入层中层功能，从而恢复对低于2.3.0版本的keras的支持。
- <strong>2019.12.14</strong>: 新增Conditional Layer Normalization及相关demo。
- <strong>2019.12.09</strong>: 各example的data_generator规范化；修复application='lm'时的一个错误。
- <strong>2019.12.05</strong>: 优化tokenizer的do_lower_case，同时微调各个example。
- <strong>2019.11.23</strong>: 将train.py重命名为optimizers.py，更新大量优化器实现，全面兼容keras和tf.keras。
- <strong>2019.11.19</strong>: 将utils.py重命名为tokenizer.py。
- <strong>2019.11.19</strong>: 想来想去，最后还是决定把snippets放到<a href="https://github.com/bojone/bert4keras/blob/master/bert4keras/snippets.py">bert4keras.snippets</a>下面去好了。
- <strong>2019.11.18</strong>: 优化预训练权重加载逻辑，增加保存模型权重至Bert的checkpoint格式方法。
- <strong>2019.11.17</strong>: <del>分离一些与Bert本身不直接相关的常用代码片段到<a href="https://github.com/bojone/python-snippets">python_snippets</a>，供其它项目共用。</del>
- <strong>2019.11.11</strong>: 添加NSP部分。
- <strong>2019.11.05</strong>: 适配<a href="https://github.com/google-research/google-research/tree/master/albert">google版albert</a>，不再支持<a href="https://github.com/brightmart/albert_zh">非Google版albert_zh</a>。
- <strong>2019.11.05</strong>: 以RoBERTa为例子的预训练代码开发完毕，同时支持TPU/多GPU训练，详见<a href="https://github.com/bojone/bert4keras/tree/master/pretraining/roberta/">roberta</a>。欢迎在此基础上构建更多的预训练代码。
- <strong>2019.11.01</strong>: 逐步增加预训练相关代码，详见<a href="https://github.com/bojone/bert4keras/tree/master/pretraining">pretraining</a>。
- <strong>2019.10.28</strong>: 支持使用基于<a href="https://github.com/google/sentencepiece">sentencepiece</a>的tokenizer。
- <strong>2019.10.25</strong>: 引入原生tokenizer。
- <strong>2019.10.22</strong>: 引入梯度累积优化器。
- <strong>2019.10.21</strong>: 为了简化代码结构，决定放弃keras 2.3.0之前的版本的支持，目前只支持keras 2.3.0+以及tf.keras。
- <strong>2019.10.20</strong>: 应网友要求，现支持直接用`model.save`保存模型结构，用`load_model`加载整个模型（只需要在`load_model`之前执行`from bert4keras.layers import *`，不需要额外写`custom_objects`）。
- <strong>2019.10.09</strong>: 已兼容tf.keras，同时在tf 1.13和tf 2.0下的tf.keras测试通过，通过设置环境变量`TF_KERAS=1`来切换tf.keras。
- <strong>2019.10.09</strong>: 已兼容Keras 2.3.x，但只是临时方案，后续可能直接移除掉2.3之前版本的支持。
- <strong>2019.10.02</strong>: 适配albert，能成功加载<a href="https://github.com/brightmart/albert_zh">albert_zh</a>的权重，只需要在`load_pretrained_model`函数里加上`albert=True`。

## 背景
之前一直用CyberZHG大佬的<a href="https://github.com/CyberZHG/keras-bert">keras-bert</a>，如果纯粹只是为了在keras下对bert进行调用和fine tune来说，keras-bert已经足够能让人满意了。

然而，如果想要在加载官方预训练权重的基础上，对bert的内部结构进行修改，那么keras-bert就比较难满足我们的需求了，因为keras-bert为了代码的复用性，几乎将每个小模块都封装为了一个单独的库，比如keras-bert依赖于keras-transformer，而keras-transformer依赖于keras-multi-head，keras-multi-head依赖于keras-self-attention，这样一重重依赖下去，改起来就相当头疼了。

所以，我决定重新写一个keras版的bert，争取在几个文件内把它完整地实现出来，减少这些依赖性，并且保留可以加载官方预训练权重的特性。

## 鸣谢
感谢CyberZHG大佬实现的<a href="https://github.com/CyberZHG/keras-bert">keras-bert</a>，本实现有不少地方参考了keras-bert的源码，在此衷心感谢大佬的无私奉献。

## 交流
QQ交流群：67729435，微信群请加机器人微信号spaces_ac_cn
